In [ ]:
%matplotlib notebook
from datetime import datetime

def pretty_print(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty_print(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))


def plot_error_distribution(errors: [float], title: str):
    plt.subplots()
    plt.title(f"Error distribution - {title}")
    plt.xlabel("Error")
    plt.ylabel("Density")
    plt.hist(errors, 50, density=True, alpha=0.75)
    plt.show() 

    
def mean_squared_error(ref: pd.DataFrame, pred: pd.DataFrame) -> float:
    return np.square(np.subtract(ref, pred)).mean().values[0]


def raw_bias(errors: [float]) -> float:
    return np.mean(errors)


def abs_raw_bias(abs_errors: [float]) -> float:
    return np.mean(abs_errors)

    
def percent_bias(ref_values: [float], pred_values: [float]) -> float:
    ref_mean = np.mean(ref_values)
    pred_mean = np.mean(pred_values)
    return 100 * abs((pred_mean - ref_mean) / ref_mean)


def sum_error(abs_errors: [float]) -> float:
    return np.sum(abs_errors)


def max_error(errors: [float]) -> float:
    res = 0
    for val in errors:
        res = val if abs(val) > res else res
    return res


def variance(df: pd.DataFrame) -> float:
    series = df.iloc[:, 0]
    mean = series.mean()
    sq_diffs = [ (it - mean) ** 2 for it in series ]
    variance = np.sum(sq_diffs) / (len(series) - 1)
    return variance


def kurtosis(df: pd.DataFrame):
    return float(df.kurtosis(skipna=True)[0])


def skewness(df: pd.DataFrame):
    return float(df.skew(skipna=True)[0])


def plot_imputation(gapped: pd.DataFrame, imputed: pd.DataFrame, title="Untitled"):
    global df, dfloader
    for target in dfloader.targets:
        fig, ax = plt.subplots(figsize=(9, 6))
        plt.title(title)
        plt.xlabel("Time")
        plt.ylabel(target)
        plt.plot(df[target], c="green", label="Reference data")
        plt.plot(imputed[target], c="red", label="Imputed data")
        plt.plot(gapped[target], c="cyan", label="Data with gaps")
        ax.legend()
        plt.show()

        fig, ax = plt.subplots(figsize=(9, 6))
        plt.title(title)
        plt.xlabel("Time")
        plt.ylabel("Error")
        plt.plot(df[target] - imputed[target], c="blue")
        plt.show()

saving_results = []    
indexes_save = []
gap_indexes_save = []
for i in range(len(imputed_dfs)):
    for target in dfloader.targets:
        flattened_indices = [it for sublist in gaps_indices[i] for it in sublist]
        ref_values = [df[[target]].iloc[:, 0][index] for index in flattened_indices]
        pred_values = [imputed_dfs[i][[target]].iloc[:, 0][index] for index in flattened_indices]

        errors = [ref_values[i] - pred_values[i] for i in range(len(ref_values))]
        abs_errors = [abs(it) for it in errors]

        title = f"{imputer['title']} with gap type {i + 1} [{dataset_config['gaps'][i][0]};{dataset_config['gaps'][i][1]}]"

        results = {
            "Mean Squared Error": mean_squared_error(df[[target]], imputed_dfs[i][[target]]),
            "Raw Bias": raw_bias(errors),
            "Absolute Raw Bias": abs_raw_bias(abs_errors),
            "Percent Bias": percent_bias(ref_values, pred_values),
            "Errors sum": sum_error(abs_errors),
            "Maximum error": max_error(errors),
            "Variance error": variance(df[[target]]) - variance(imputed_dfs[i][[target]]),
            "Kurtosis error": kurtosis(df[[target]]) - kurtosis(imputed_dfs[i][[target]]),
            "Skewness error": skewness(df[[target]]) - skewness(imputed_dfs[i][[target]]),
        }

        #saving the results
        gap_indexes_save.append(f"Data with gap type {i + 1} [{dataset_config['gaps'][i][0]};{dataset_config['gaps'][i][1]}]")
        indexes_save.append(title)
        saving_results.append(results)

        print(title)
        pretty_print(results, indent=1)
        plot_imputation(dfs_with_gaps[i][[target]], imputed_dfs[i][[target]], f"{imputer['title']} with gap type {i + 1}")
        print("\n\n\n")
